# TV Script Generation

In [1]:
import helper
import numpy as np
import tensorflow as tf
from collections import Counter

data_dir = './data/simpsons/moes_tavern_lines.txt'
text = helper.load_data(data_dir)
# Ignore notice, since we don't use it for analysing the data
text = text[81:]

In [2]:
# Explore the Data

view_sentence_range = (0, 10)

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))
scenes = text.split('\n\n')
print('Number of scenes: {}'.format(len(scenes)))
sentence_count_scene = [scene.count('\n') for scene in scenes]
print('Average number of sentences in each scene: {}'.format(np.average(sentence_count_scene)))

sentences = [sentence for scene in scenes for sentence in scene.split('\n')]
print('Number of lines: {}'.format(len(sentences)))
word_count_sentence = [len(sentence.split()) for sentence in sentences]
print('Average number of words in each line: {}'.format(np.average(word_count_sentence)))

print()
print('The sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 11492
Number of scenes: 262
Average number of sentences in each scene: 15.248091603053435
Number of lines: 4257
Average number of words in each line: 11.50434578341555

The sentences 0 to 10:
Moe_Szyslak: (INTO PHONE) Moe's Tavern. Where the elite meet to drink.
Bart_Simpson: Eh, yeah, hello, is Mike there? Last name, Rotch.
Moe_Szyslak: (INTO PHONE) Hold on, I'll check. (TO BARFLIES) Mike Rotch. Mike Rotch. Hey, has anybody seen Mike Rotch, lately?
Moe_Szyslak: (INTO PHONE) Listen you little puke. One of these days I'm gonna catch you, and I'm gonna carve my name on your back with an ice pick.
Moe_Szyslak: What's the matter Homer? You're not your normal effervescent self.
Homer_Simpson: I got my problems, Moe. Give me another one.
Moe_Szyslak: Homer, hey, you should not drink to forget your problems.
Barney_Gumble: Yeah, you should only drink to enhance your social skills.




In [1]:
def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    word_counts  = Counter(text)
    sorted_vocab = sorted(word_counts, key=word_counts.get, reverse=True)
    int_to_vocab = {ii: word for ii, word in enumerate(sorted_vocab)}
    vocab_to_int = {word: ii for ii, word in int_to_vocab.items()}
    
    return vocab_to_int, int_to_vocab

def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenize dictionary where the key is the punctuation and the value is the token
    """
    tokenize_dictionary = {
        '.':  '||Period||',
        ',':  '||Comma||',
        '"':  '||Quotation_Mark||',
        ';':  '||Semicolon||',
        '!':  '||Exclamation_mark||',
        '?':  '||Question_mark||',
        '(':  '||Left_Parentheses||',
        ')':  '||Right_Parentheses||',
        '--': '||Dash||',
        "\n": '||Return||'
    }

    return tokenize_dictionary

In [5]:
# Preprocess Training, Validation, and Testing Data
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_table
                                s)
int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

In [8]:
def get_inputs():
    """
    Create TF Placeholders for input, targets, and learning rate.
    :return: Tuple (input, targets, learning rate)
    """
    input         = tf.placeholder(tf.int32, [None, None], name='input')
    target        = tf.placeholder(tf.int32, [None, None], name='target')
    learning_rate = tf.placeholder(tf.float32, name='learning_rate')    

    return input, target, learning_rate

def get_init_cell(batch_size, rnn_size):
    """
    Create an RNN Cell and initialize it.
    :param batch_size: Size of batches
    :param rnn_size: Size of RNNs
    :return: Tuple (cell, initialize state)
    """
    # Define number of LSTM layers
    number_of_layers = 1
    # Keep probability
    keep_probability = 0.8
    # Basic LSTM cell
    lstm = tf.contrib.rnn.BasicLSTMCell(rnn_size)
    # Add dropout
    drop = tf.contrib.rnn.DropoutWrapper(lstm, keep_probability)
    # Miltiple LSTM layers
    cell = tf.contrib.rnn.MultiRNNCell([drop] * number_of_layers)
    # Initialize state of all zeros
    initial_state_of_zeros = cell.zero_state(batch_size, tf.float32)
    # Initialize tensor 
    initial_state = tf.identity(initial_state_of_zeros, name="initial_state")
    
    return cell, initial_state

def get_embed(input_data, vocab_size, embed_dim):
    """
    Create embedding for <input_data>.
    :param input_data: TF placeholder for text input.
    :param vocab_size: Number of words in vocabulary.
    :param embed_dim: Number of embedding dimensions
    :return: Embedded input.
    """
    embedding = tf.Variable(tf.random_uniform((vocab_size, embed_dim), -1, 1))
    # Lookup the list of tensors
    embed     = tf.nn.embedding_lookup(embedding, input_data)
    
    return embed

def build_rnn(cell, inputs):
    """
    Create a RNN using a RNN Cell
    :param cell: RNN Cell
    :param inputs: Input text data
    :return: Tuple (Outputs, Final State)
    """
    # Create the RNN
    outputs, final_state = tf.nn.dynamic_rnn(cell, inputs, dtype=tf.float32)
    # Lookup the list of tensors    
    final_state          = tf.identity(final_state, 'final_state')
    
    return outputs, final_state

def build_nn(cell, rnn_size, input_data, vocab_size):
    """
    Build part of the neural network
    :param cell: RNN cell
    :param rnn_size: Size of rnns
    :param input_data: Input data
    :param vocab_size: Vocabulary size
    :return: Tuple (Logits, FinalState)
    """
    # Embeding to the input data
    embed = get_embed(input_data, vocab_size, rnn_size)
    # Build the RNN
    outputs, final_state = build_rnn(cell, embed)
    # Fully connected layer
    logits = tf.contrib.layers.fully_connected(outputs, vocab_size, activation_fn=None)
    
    return logits, final_state

def get_batches(int_text, batch_size, seq_length):
    """
    Return batches of input and target
    :param int_text: Text with the words replaced by their ids
    :param batch_size: The size of batch
    :param seq_length: The length of sequence
    :return: Batches as a Numpy array
    """
    n_batches = len(int_text) // (batch_size * seq_length)
    x         = np.array(int_text[: n_batches * batch_size * seq_length])
    y         = np.array(int_text[1: n_batches * batch_size * seq_length + 1])
    
    batches   = np.array(list(zip(
        build_batch(x, batch_size, n_batches),
        build_batch(y, batch_size, n_batches)
    )))
    
    return batches

def build_batch(input, batch_size, n_batches):
    return np.split(input.reshape(batch_size, -1), n_batches, 1)

Tests Passed


In [2]:
# Hyperparameters

# Number of Epochs
num_epochs = 50
# Batch Size
batch_size = 256
# RNN Size
rnn_size = 512
# Sequence Length
seq_length = 20
# Learning Rate
learning_rate = 0.01
# Show stats for every n number of batches
show_every_n_batches = 100

In [145]:
# Build the Graph
from tensorflow.contrib import seq2seq

train_graph = tf.Graph()
with train_graph.as_default():
    vocab_size = len(int_to_vocab)
    input_text, targets, lr = get_inputs()
    input_data_shape = tf.shape(input_text)
    cell, initial_state = get_init_cell(input_data_shape[0], rnn_size)
    logits, final_state = build_nn(cell, rnn_size, input_text, vocab_size)

    # Probabilities for generating words
    probs = tf.nn.softmax(logits, name='probs')

    # Loss function
    cost = seq2seq.sequence_loss(
        logits,
        targets,
        tf.ones([input_data_shape[0], input_data_shape[1]]))

    # Optimizer
    optimizer = tf.train.AdamOptimizer(lr)

    # Gradient Clipping
    gradients = optimizer.compute_gradients(cost)
    capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients]
    train_op = optimizer.apply_gradients(capped_gradients)

In [146]:
# Train model
batches = get_batches(int_text, batch_size, seq_length)

with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(num_epochs):
        state = sess.run(initial_state, {input_text: batches[0][0]})

        for batch_i, (x, y) in enumerate(batches):
            feed = {
                input_text: x,
                targets: y,
                initial_state: state,
                lr: learning_rate}
            train_loss, state, _ = sess.run([cost, final_state, train_op], feed)

            # Show every <show_every_n_batches> batches
            if (epoch_i * len(batches) + batch_i) % show_every_n_batches == 0:
                print('Epoch {:>3} Batch {:>4}/{}   train_loss = {:.3f}'.format(
                    epoch_i,
                    batch_i,
                    len(batches),
                    train_loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_dir)
    print('Model Trained and Saved')

Epoch   0 Batch    0/13   train_loss = 8.820
Epoch   7 Batch    9/13   train_loss = 2.812
Epoch  15 Batch    5/13   train_loss = 1.566
Epoch  23 Batch    1/13   train_loss = 0.904
Epoch  30 Batch   10/13   train_loss = 0.587
Epoch  38 Batch    6/13   train_loss = 0.449
Epoch  46 Batch    2/13   train_loss = 0.369
Model Trained and Saved


In [147]:
# Save parameters for checkpoint
helper.save_params((seq_length, save_dir))

In [148]:
# Checkpoint
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
seq_length, load_dir = helper.load_params()

In [3]:
# Gererate functions

def get_tensors(loaded_graph):
    """
    Get input, initial state, final state, and probabilities tensor from <loaded_graph>
    :param loaded_graph: TensorFlow graph loaded from file
    :return: Tuple (InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)
    """
    # Get all required tensors
    input_tensor  = loaded_graph.get_tensor_by_name("input:0")
    initial_state = loaded_graph.get_tensor_by_name("initial_state:0")
    final_state   = loaded_graph.get_tensor_by_name("final_state:0")
    probabilities = loaded_graph.get_tensor_by_name("probs:0")
    
    return input_tensor, initial_state, final_state, probabilities

def pick_word(probabilities, int_to_vocab):
    """
    Pick the next word in the generated text
    :param probabilities: Probabilites of the next word
    :param int_to_vocab: Dictionary of word ids as the keys and words as the values
    :return: String of the predicted word
    """
    return int_to_vocab[np.argmax(probabilities)]

In [155]:
# Generate TV Script
gen_length = 300
# homer_simpson, moe_szyslak, or Barney_Gumble
prime_word = 'homer_simpson'


loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_dir + '.meta')
    loader.restore(sess, load_dir)

    # Get Tensors from loaded model
    input_text, initial_state, final_state, probs = get_tensors(loaded_graph)

    # Sentences generation setup
    gen_sentences = [prime_word + ':']
    prev_state = sess.run(initial_state, {input_text: np.array([[1]])})

    # Generate sentences
    for n in range(gen_length):
        # Dynamic Input
        dyn_input = [[vocab_to_int[word] for word in gen_sentences[-seq_length:]]]
        dyn_seq_length = len(dyn_input[0])

        # Get Prediction
        probabilities, prev_state = sess.run(
            [probs, final_state],
            {input_text: dyn_input, initial_state: prev_state})
        
        pred_word = pick_word(probabilities[dyn_seq_length-1], int_to_vocab)

        gen_sentences.append(pred_word)
    
    # Remove tokens
    tv_script = ' '.join(gen_sentences)
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        tv_script = tv_script.replace(' ' + token.lower(), key)
    tv_script = tv_script.replace('\n ', '\n')
    tv_script = tv_script.replace('( ', '(')
        
    print(tv_script)

homer_simpson: hey everybody, i'm peter pantsless!(laughs)
barney_gumble:(pained moan) i can't take this!
chief_wiggum:(pissed) fine, it is. / canyonero....
announcer:(quickly) federal highway commission has ruled the canyonero unsafe for me though.
moe_szyslak: so who would you like to be a snitch?
moe_szyslak: uh, yeah. and ya know what really aggravazes me, then i could do it, huh?
moe_szyslak: sure thing, mouse pad. ten bucks. that's how i do, but not me to come clean and...(loud) hey, how do you keep your help!
little_man:(to camera) that seems nice.
moe_szyslak:(lowers gun) but, can you blame me?
bart_simpson:(relieved noise, then) there's more secret than i thought!


moe_szyslak: hey, homer.
homer_simpson:(looking at watch) ah. finished with fifteen seconds to empty, barn.
ned_flanders:(surprised) you all wanna be sick tonight.
barney_gumble: hey moe, you wanna smell my flower, eh? yeah.(points) except for me.
moe_szyslak:(pained moan) i can't take this alone, kid. and now this